# Cleaning and Aspect Detection

In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk
import string
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch
from textblob import TextBlob
from tqdm.notebook import tqdm
from symspellpy import SymSpell, Verbosity
import pkg_resources
tqdm.pandas()
pd.set_option('display.max_colwidth', None)
from pyabsa import ATEPCCheckpointManager
from collections import defaultdict


/var/folders/d7/grxt144d6wz6r6mqxx6jmqwm0000gn/T/ipykernel_70518/3528560753.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


No CUDA GPU found in your device


/bin/sh: nvidia-smi: command not found


[2025-04-28 10:38:34] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






/Users/mariecieslar/miniforge3/envs/nlp/lib/python3.11/multiprocessing/pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


## Goal:
In this file we perform some data cleaning of our review data, we compare how applying pyabsa to different 

In [2]:
df = pd.read_csv('filtered_england_recent.csv')

## Data sampling and cleaning
Working with a sample size of 1% to test cleaning techniques before applying pyabsa

In [3]:
# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Sample 5% of the data 
df_sample = df.sample(frac=0.01, random_state=21).copy()
def clean_min(text):
    if not isinstance(text, str):
        return ""
    return text.replace('\n', ' ').replace('\r', '').encode("ascii", errors="ignore").decode().strip()

#only lematizing, want to keep as many words as possible. 
def preprocess_text_lemmaonly(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text.lower())
    return " ".join(token.lemma_ for token in doc if not token.is_punct and not token.is_space)

def preprocess_text_with_stopwords(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text.lower())
    return " ".join(
        token.lemma_ 
        for token in doc 
        if not token.is_punct 
        and not token.is_space 
        and not token.is_stop
    )

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
def correct_token(token_text):
    suggestions = sym_spell.lookup(token_text, Verbosity.CLOSEST, max_edit_distance=2)
    if suggestions:
        return suggestions[0].term
    return token_text

def preprocess_text_symspell(text):
    if pd.isnull(text):
        return ""
    corrected_tokens = []
    for token in nlp(text.lower()):
        if token.is_punct or token.is_space or token.like_num or token.is_stop or len(token) < 3:
            continue
        corrected = correct_token(token.text)
        doc = nlp(corrected)  # lemmatize after correction
        for sub_token in doc:
            if not sub_token.is_punct and not sub_token.is_space:
                corrected_tokens.append(sub_token.lemma_)
    return " ".join(corrected_tokens)


# Apply preprocessing with progress bar
for col in ['pros', 'cons']:
    print(f"Processing column: {col}")
    df_sample[f'{col}_clean_min'] = df_sample[col].progress_apply(clean_min)
    df_sample[f'{col}_clean_lemma_only'] = df_sample[col].progress_apply(preprocess_text_lemmaonly)
    df_sample[f'{col}_clean_with_stopwords'] = df_sample[col].progress_apply(preprocess_text_with_stopwords)
    df_sample[f'{col}_clean_with_symspell'] = df_sample[col].progress_apply(preprocess_text_symspell)


# Preview result
df_sample.head()

Processing column: pros


  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

...

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

,Unnamed: 0,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,...,cons,year,pros_clean_min,pros_clean_lemma_only,pros_clean_with_stopwords,pros_clean_with_symspell,cons_clean_min,cons_clean_lemma_only,cons_clean_with_stopwords,cons_clean_with_symspell
77921,755273,Taylor Wimpey,2020-05-26,Design & Planning Executive,Current Employee,"Solihull, England, England",5,5.0,5.0,NaN,...,Nothing to report on cons,2020,"Through all the uncertainty recently, the business has given us regular updates and so much help to our subcontractors as well.",through all the uncertainty recently the business have give we regular update and so much help to our subcontractor as well,uncertainty recently business give regular update help subcontractor,uncertainty recently business give regular update help subcontractor,Nothing to report on cons,nothing to report on con,report con,report con
69618,704703,RSM,2020-10-29,Tasting Room Attendant,"Current Employee, more than 8 years","London, England, England",1,NaN,1.0,1.0,...,Didn't learn anything and some staff got the privilege and got more right than ten years of pratical working experience,2020,10 years enough of working,10 year enough of work,10 year work,year work,Didn't learn anything and some staff got the privilege and got more right than ten years of pratical working experience,do not learn anything and some staff get the privilege and get more right than ten year of pratical working experience,learn staff get privilege get right year pratical working experience,learn staff get privilege get right year pratical work experience
60239,603936,NHS,2019-04-01,Health Care Assistant,"Former Employee, more than 3 years","Huddersfield, England, England",1,1.0,3.0,NaN,...,Terrible place to work\r\nDon't go there,2019,None terrible place Don't go there,none terrible place do not go there,terrible place,terrible place,Terrible place to work Don't go there,terrible place to work do not go there,terrible place work,terrible place work
74473,742994,Savills,2018-12-12,Associate,"Current Employee, more than 1 year","London, England, England",4,5.0,5.0,NaN,...,"As it is such a big company, process can slow down getting things done.\nLower salaries at some levels due to bonus potential.",2018,Great people. The open plan culture means that everyone is accessible. Gorgeous office in a great location. Work/life balance. Flexible/home working possible as their IT systems are so good. Internal movement is encouraged.,great people the open plan culture mean that everyone be accessible gorgeous office in a great location work life balance flexible home working possible as their it system be so good internal movement be encourage,great people open plan culture mean accessible gorgeous office great location work life balance flexible home working possible system good internal movement encourage,great people open plan culture mean accessible gorgeous office great location work life balance flexible home work possible system good internal movement encourage,"As it is such a big company, process can slow down getting things done. Lower salaries at some levels due to bonus potential.",as it be such a big company process can slow down get thing do low salary at some level due to bonus potential,big company process slow get thing low salary level bonus potential,big company process slow get thing lower salary level bonus potential
7099,56504,B and Q,2017-10-31,Customer Service Advisor,"Current Employee, more than 10 years","London, England, England",2,1.0,1.0,NaN,...,Management\r\nH/R\r\nBack stabbing\r\nLack of professionalism\r\nTraining,2017,Holidays Flexibility Staff discount Colleagues Customer diversity,holiday flexibility staff discount colleague customer diversity,holiday flexibility staff discount colleague customer diversity,holiday flexibility staff discount colleague customer diversity,Management H/R Back stabbing Lack of professionalism Training,management

In [4]:
df_sample[['pros', 'cons', 'pros_clean_lemma_only', 'pros_clean_with_stopwords', 'pros_clean_with_symspell', 'cons_clean_lemma_only','cons_clean_with_stopwords', 'cons_clean_with_symspell']].head()

,pros,cons,pros_clean_lemma_only,pros_clean_with_stopwords,pros_clean_with_symspell,cons_clean_lemma_only,cons_clean_with_stopwords,cons_clean_with_symspell
77921,"Through all the uncertainty recently, the business has given us regular updates and so much help to our subcontractors as well.",Nothing to report on cons,through all the uncertainty recently the business have give we regular update and so much help to our subcontractor as well,uncertainty recently business give regular update help subcontractor,uncertainty recently business give regular update help subcontractor,nothing to report on con,report con,report con
69618,10 years enough of working,Didn't learn anything and some staff got the privilege and got more right than ten years of pratical working experience,10 year enough of work,10 year work,year work,do not learn anything and some staff get the privilege and get more right than ten year of pratical working experience,learn staff get privilege get right year pratical working experience,learn staff get privilege get right year pratical work experience
60239,None terrible place\r\nDon't go there,Terrible place to work\r\nDon't go there,none terrible place do not go there,terrible place,terrible place,terrible place to work do not go there,terrible place work,terrible place work
74473,Great people.\nThe open plan culture means that everyone is accessible.\nGorgeous office in a great location.\nWork/life balance.\nFlexible/home working possible as their IT systems are so good.\nInternal movement is encouraged.,"As it is such a big company, process can slow down getting things done.\nLower salaries at some levels due to bonus potential.",great people the open plan culture mean that everyone be accessible gorgeous office in a great location work life balance flexible home working possible as their it system be so good internal movement be encourage,great people open plan culture mean accessible gorgeous office great location work life balance flexible home working possible system good internal movement encourage,great people open plan culture mean accessible gorgeous office great location work life balance flexible home work possible system good internal movement encourage,as it be such a big company process can slow down get thing do low salary at some level due to bonus potential,big company process slow get thing low salary level bonus potential,big company process slow get thing lower salary level bonus potential
7099,Holidays \r\nFlexibility\r\nStaff discount\r\nColleagues \r\nCustomer diversity,Management\r\nH/R\r\nBack stabbing\r\nLack of professionalism\r\nTraining,holiday flexibility staff discount colleague customer diversity,holiday flexibility staff discount colleague customer diversity,holiday flexibility staff discount colleague customer diversity,management h r back stab lack of professionalism training,management h r stab lack professionalism training,management stab lack professionalism train


## ABSA

In [5]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
def get_aspect_sentiment(text, aspect, tone):
    if pd.isnull(text):
        return None

    # Compose prompt-like input for aspect analysis
    if tone == 'pos':
        combined = f"The following is a positive experience about {aspect}: {text}"
    else:
        combined = f"The following is a negative experience about {aspect}: {text}"

    inputs = tokenizer(combined, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        logits = model(**inputs).logits
    scores = softmax(logits.numpy()[0])

    # This model returns scores for: [1 star, 2 stars, 3 stars, 4 stars, 5 stars]
    star_labels = [1, 2, 3, 4, 5]
    result = dict(zip(star_labels, scores))
    result["label"] = star_labels[scores.argmax()]
    return result


In [7]:
from collections import Counter
failed_sentiment = Counter()

def analyze_all_aspect_groups(text, aspect_groups, tone):
    results = {}
    for aspect, keywords in aspect_groups.items():
        if any(kw in text.lower() for kw in keywords):
            sent = get_aspect_sentiment(text, aspect, tone)
            if sent:
                results[aspect] = sent
            else:
                failed_sentiment[aspect] += 1  # Track failures
    return results


## Aspect detection
Using automatic aspect detection to find what aspects get mentioned the most in the reviews

In [8]:
# Load ABSA model
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(
    checkpoint='english',
    auto_device=True
)

# Sample for ABSA (adjust size as needed)
texts_pros = df_sample['pros_clean_min'].dropna().sample(frac=0.5, random_state=42).tolist()
texts_cons = df_sample['cons_clean_min'].dropna().sample(frac=0.5, random_state=42).tolist()

# Extract aspects from pros and cons
absa_results_pros = aspect_extractor.extract_aspect(inference_source=texts_pros, print_result=False)
absa_results_cons = aspect_extractor.extract_aspect(inference_source=texts_cons, print_result=False)

# Collect discovered aspects
from collections import Counter

aspect_counter = Counter()
for result in absa_results_pros + absa_results_cons:
    for asp in result.get("aspect", []):
        aspect_counter[asp.lower()] += 1

# Show top discovered aspects
top_aspects = [asp for asp, count in aspect_counter.items() if count >= 3]
print(f"Top discovered aspects: {top_aspects}")


[2025-04-28 10:39:34] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-28 10:39:34] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-28 10:39:34] (2.4.1.post1) Downloading checkpoint:english 
[2025-04-28 10:39:34] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-04-28 10:39:34] (2.4.1.post1) Checkpoint already downloaded, skip
[2025-04-28 10:39:34] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2025-04-28 10:39:34] (2.4.1.post1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2025-04-28 10:39:34] (2.4.1.post1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apc

/Users/mariecieslar/miniforge3/envs/nlp/lib/python3.11/site-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

[2025-04-28 10:41:33] (2.4.1.post1) The results of aspect term extraction have been saved in /Users/mariecieslar/Desktop/Schreibtisch - Maries MacBook Air/Bocconi/Semester 2/NLP/Final Project/glassdoor_reviews/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json


classifying aspect sentiments: 100%|██████████| 14/14 [00:51<00:00,  3.71s/it]


[2025-04-28 10:43:16] (2.4.1.post1) The results of aspect term extraction have been saved in /Users/mariecieslar/Desktop/Schreibtisch - Maries MacBook Air/Bocconi/Semester 2/NLP/Final Project/glassdoor_reviews/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
Top discovered aspects: ['training', 'projects', 'people', 'development', 'pay', 'environment', 'culture', 'team', 'hours', 'management', 'pension', 'staff', 'pace', 'food', 'drinks', 'colleagues', 'company', 'politics', 'lunch', 'tea', 'pros', 'canteen', 'benefits', 'teams', 'manager', 'support', 'rules', 'salary', 'managers', 'values', 'holidays', 'atmosphere', 'job', 'office', 'location', 'leadership', 'flexibility', 'place', 'flexible', 'travel', 'shift', 'service', 'employees', 'money', 'shifts', 'opportunities', 'learn', 'offices', 'facilities', 'perks', 'discount', 'everyone', 'work', 'progression', 'workload', 'career', 'experience', 'organisation', 'commission', 'bonuses', 'communication', 'day

In [9]:
from collections import Counter

# Combine pros and cons into one string per row
combined_texts = (
    df_sample['pros_clean_min'].fillna('') + ' ' + df_sample['cons_clean_min'].fillna('')
).str.lower()

# Count how many unique reviews mention each aspect
aspect_review_counts = {asp: combined_texts.str.contains(rf'\b{asp.lower()}\b').sum() for asp in top_aspects}

# Display sorted
print("\n🧾 Number of Reviews Mentioning Each Auto-Detected Aspect (Unique Mentions Only):")
for asp, count in sorted(aspect_review_counts.items(), key=lambda x: -x[1]):
    print(f"{asp:<15}: {count}")



🧾 Number of Reviews Mentioning Each Auto-Detected Aspect (Unique Mentions Only):
work           : 345
people         : 220
management     : 170
company        : 156
pay            : 148
hours          : 145
staff          : 137
environment    : 116
team           : 114
opportunities  : 112
job            : 102
benefits       : 100
culture        : 95
progression    : 87
flexible       : 84
training       : 80
colleagues     : 74
career         : 70
office         : 63
support        : 61
managers       : 61
salary         : 60
place          : 59
employees      : 56
development    : 47
experience     : 44
organisation   : 37
teams          : 33
projects       : 32
atmosphere     : 32
everyone       : 32
manager        : 31
money          : 30
flexibility    : 29
shifts         : 29
location       : 27
learn          : 27
communication  : 26
leadership     : 25
discount       : 25
travel         : 24
customers      : 24
service        : 23
days           : 22
pension        : 21
perks 

## Manual aspects
Came up with a list of aspects candidates may value

In [10]:
aspect_groups = {
    "pay": ["pay", "salary", "compensation", "wage", "income", "bonus"],
    "management": ["management", "managers", "leadership", "boss", "supervisor"],
    "work-life balance": ["work-life balance", "balance", "working hours", "hours", "flexibility", "overtime", "holiday"],
    "growth": ["growth", "grow", "career", "promotion", "advancement", "development", "opportunity", "progress"],
    "stress & pressure": ["stress", "pressure", "burnout", "demanding", "high pressure", "overload", "long hours", "stressful"],
    "culture": ["culture", "environment", "atmosphere", "company culture", "values"],
    "diversity & inclusion": ["diversity", "inclusion", "equality", "equity", "inclusive", "minority", "belonging"],
    "travel": ["travel", "commute", "commuting", "relocation", "mobility"],
    "benefits": ["benefits", "perks", "healthcare", "insurance", "pension", "rewards", "gym", "lunch", "dental"],
    "remote work": ["remote", "work from home", "wfh", "home office", "hybrid", "telework"],
    "flexibility": ["flexible", "flexibility", "adjustable schedule", "shift", "adaptable"],
    "people": ["people", "colleagues", "collegues", "coworker", "coworkers", "community", "teammates"],
    "training & onboarding": ["training", "onboarding", "orientation", "new hire", "introduction", "induction"],
    "job role clarity": ["role", "responsibility", "job description", "tasks", "duties", "expectation", "scope"],
    "tools & resources": ["tools", "resources", "equipment", "software", "support", "access", "tech stack", "technology"],
    "performance feedback": ["feedback", "review", "appraisal", "evaluation", "performance", "supervision", "goals"],
    "job security": ["job security", "layoff", "firing", "termination", "redundancy", "stable job"],
    "teamwork & collaboration": ["team", "teamwork", "collaboration", "supportive", "helpful", "team spirit", "group"]
}
def safe_analyze(text, aspect_groups, tone):
    try:
        return analyze_all_aspect_groups(text, aspect_groups, tone)
    except Exception as e:
        print(f"Failed on: {repr(text)}\nError: {e}")
        return {}

df_sample['pro_aspect_sentiments'] = df_sample['pros_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, aspect_groups, tone='pos'))
df_sample['con_aspect_sentiments'] = df_sample['cons_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, aspect_groups, tone='neg'))

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

In [11]:
# Full list of all aspect labels from your aspect_groups
all_aspects = list(aspect_groups.keys())
pro_counts= Counter()
con_counts = Counter()
pro_counts_clean = Counter()
con_counts_clean = Counter()

for row in df_sample['pro_aspect_sentiments'].dropna():
    pro_counts.update(row.keys())
for row in df_sample['con_aspect_sentiments'].dropna():
    con_counts.update(row.keys())

# Ensure all aspects appear, even if 0
pro_data = [(aspect, pro_counts.get(aspect, 0)) for aspect in all_aspects]
con_data = [(aspect, con_counts.get(aspect, 0)) for aspect in all_aspects]

# Sort by count
pro_data.sort(key=lambda x: x[1])
con_data.sort(key=lambda x: x[1])

# Ensure all aspects appear, even if 0
pro_data_clean = [(aspect, pro_counts_clean.get(aspect, 0)) for aspect in all_aspects]
con_data_clean = [(aspect, con_counts_clean.get(aspect, 0)) for aspect in all_aspects]

# Sort by count
pro_data_clean.sort(key=lambda x: x[1])
con_data_clean.sort(key=lambda x: x[1])

## Ultimate aspect selection
after running automatic aspect detection, we grouped aspects into categories based on results and our own knowledge of company aspects that are important to employees and candidates

In [12]:
# 1. Normalise everything to lowercase
existing_keys = {k.lower() for k in aspect_groups.keys()}

# flatten every keyword in the dictionary
existing_keywords = {kw.lower() for kws in aspect_groups.values() for kw in kws}

# 2. Compare
# a) aspects that are missing entirely
missing_new_keys = [asp for asp in top_aspects
                    if asp.lower() not in existing_keys
                    and asp.lower() not in existing_keywords]

# b) aspects that already exist as *keywords* but not as a top‑level key
present_as_keyword = [asp for asp in top_aspects
                      if asp.lower() not in existing_keys    # not a key
                      and asp.lower() in existing_keywords]  # but inside values

print("🔍 Candidates to ADD as new keys:")
for asp in missing_new_keys:
    print("  •", asp)

print("\n✅ Already present as a keyword (maybe promote to key?):")
for asp in present_as_keyword:
    print("  •", asp)

🔍 Candidates to ADD as new keys:
  • projects
  • staff
  • pace
  • food
  • drinks
  • company
  • politics
  • tea
  • pros
  • canteen
  • teams
  • manager
  • rules
  • holidays
  • job
  • office
  • location
  • place
  • service
  • employees
  • money
  • shifts
  • opportunities
  • learn
  • offices
  • facilities
  • discount
  • everyone
  • work
  • progression
  • workload
  • experience
  • organisation
  • commission
  • bonuses
  • communication
  • days
  • targets
  • bureaucracy
  • tape
  • customers
  • processes
  • it systems

✅ Already present as a keyword (maybe promote to key?):
  • training
  • development
  • environment
  • team
  • hours
  • pension
  • colleagues
  • lunch
  • support
  • salary
  • managers
  • values
  • atmosphere
  • leadership
  • flexible
  • shift
  • perks
  • career
  • promotion


### Combining for ultimate dictionary:

In [13]:
modified_aspect_groups = {
    'perks & benefits': [
        'benefit', 'benefits', 'employee benefits', 'perks', 'health',
        'healthcare', 'insurance', 'pension', 'pension scheme', 'rewards',
        'gym', 'lunch', 'dental', 'benefits package', 'package', 'nhs',
        'stock', 'discount', 'discounts', 'food', 'coffee', 'drink',
        'drinks', 'meal', 'meals'],

    'culture': [
        'culture', 'family', 'environment', 'atmosphere', 'company culture',
        'values', 'enviroment', 'work environment', 'office environment',
        'morale', 'communication', 'communications', 'vision', 'sustainability''attitude',
        'bullying', 'interaction', 'gossip', 'dress code'],

    'diversity & inclusion': [
        'diversity', 'inclusion', 'equality', 'equity', 'inclusive',
        'minority', 'belonging'],

    'growth & learning': [
        'growth', 'grow', 'career', 'promotion', 'promotions', 'advancement',
        'development', 'opportunity', 'progress', 'opportunities', 'progression',
        'career progression', 'learn', 'learning', 'knowledge', 'courses', 'academics',
        'academic', 'academy', 'student', 'students', 'teaching', 'university', 'internal movement'],

    'hr & recruitment': [
        'hr', 'recruitment', 'talent', 'staff', 'staff members', 'staffing',
        'policies', 'training', 'onboarding', 'orientation', 'new hire', 'introduction',
        'induction'],

    'job role clarity': [
        'role', 'responsibility', 'job description', 'tasks', 'duties',
        'expectation', 'scope'],

    'job security': [
        'job security', 'security', 'safety', 'layoff', 'firing', 'termination',
        'redundancy', 'stable job', 'employment', 'staff turnover', 'turnover'],

    'management': [
        'directors', 'management', 'managers', 'leader', 'leaders',
        'leadership', 'boss', 'supervisor', 'supervisors', 'ceo', 'bosses',
        'management structure', 'management team', 'middle management',
        'senior management', 'structure', 'bureaucracy', 'bureaucratic',
        'red tape', 'tape', 'politics'],

    'office & location': [
        'office', 'office location', 'office space',
        'offices', 'location', 'locations', 'area', 'workplace',
        'building', 'buildings', 'facilities', 'space', 'room',
        'infrastructure', 'campus', 'canteen', 'parking'],

    'pay': [
        'pay', 'paid', 'money', 'salary', 'compensation', 'wage', 'wages',
        'income', 'bonus', 'bonuses', 'reward', 'salaries', 'sick pay',
        'incentives', 'pay rates', 'commission', 'contract',
        'contracts', 'terms', 'conditions'],

    'people': [
        'people', 'colleagues', 'collegues', 'coworker', 'coworkers', 'community',
        'teammates', 'social', 'crew'],

    'performance feedback': [
        'feedback', 'review', 'appraisal', 'evaluation', 'performance',
        'supervision', 'goals', 'targets', 'kpi'],

    'remote work': [
        'remote', 'work from home', 'wfh', 'home office', 'hybrid', 'telework'],

    'stress & pressure': [
        'stress', 'paced environment', 'pace', 'pressure', 'burnout',
        'demanding', 'high pressure', 'overload', 'long hours', 'stressful'],

    'teamwork & collaboration': [
        'team', 'teams', 'teamwork', 'collaboration', 'supportive',
        'helpful', 'team spirit', 'group'],

    'tools & resources': [
        'tools', 'resources', 'equipment', 'software', 'support', 'access',
        'tech stack', 'technology', 'technology', 'tech', 'innovation',
        'technologies', 'research'],

    'work-life balance': [
        'work-life balance', 'workload', 'weekends', 'balance', 'working hours',
        'hours', 'flexibility', 'overtime', 'holiday', 'life balance', 'lifestyle',
        'annual leave', 'christmas', 'flexible', 'adjustable schedule', 'shift', 'adaptable',
        'break', 'breaks']
}

### Analyzing which reviews the aspect detection works best on
comparing clean min, clean lemma only, and clean____

In [14]:
df_sample['min_pro_aspect_sentiments'] = df_sample['pros_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['min_con_aspect_sentiments'] = df_sample['cons_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))
df_sample['lemma_pro_aspect_sentiments'] = df_sample['pros_clean_lemma_only'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['lemma_con_aspect_sentiments'] = df_sample['cons_clean_lemma_only'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))
df_sample['Stopword_pro_aspect_sentiments'] = df_sample['pros_clean_with_stopwords'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['Stopword_con_aspect_sentiments'] = df_sample['cons_clean_with_stopwords'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))
df_sample['Symspell_pro_aspect_sentiments'] = df_sample['pros_clean_with_symspell'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['Symspell_con_aspect_sentiments'] = df_sample['cons_clean_with_symspell'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

In [15]:
df_sample.to_csv('comparison_cleans.csv')

In [16]:
df_sample[['firm', 'pros', 'cons',
       'pros_clean_min', 'pros_clean_lemma_only',
       'pros_clean_with_stopwords', 'pros_clean_with_symspell',
       'cons_clean_min', 'cons_clean_lemma_only', 'cons_clean_with_stopwords',
       'cons_clean_with_symspell', 'min_pro_aspect_sentiments','lemma_pro_aspect_sentiments','Stopword_pro_aspect_sentiments','Symspell_pro_aspect_sentiments',
       'min_con_aspect_sentiments', 
       'lemma_con_aspect_sentiments', 
       'Stopword_con_aspect_sentiments', 
       'Symspell_con_aspect_sentiments']].to_csv('comparison_cleans.csv')

In [17]:
from collections import defaultdict

# Define clean types and columns
clean_types = ['min', 'lemma', 'Stopword', 'Symspell']
pro_cols = [f'{c}_pro_aspect_sentiments' for c in clean_types]
con_cols = [f'{c}_con_aspect_sentiments' for c in clean_types]

# Collect all aspects seen in the data
all_aspects = sorted({
    aspect
    for col in pro_cols + con_cols
    for entry in df_sample[col].dropna()
    for aspect in entry.keys()  # no need to eval, it's already a dict
})

# Count mentions in pros
pro_counts = defaultdict(dict)
for method in clean_types:
    col = f'{method}_pro_aspect_sentiments'
    for aspect in all_aspects:
        pro_counts[aspect][method] = df_sample[col].dropna().apply(lambda x: aspect in x).sum()

# Count mentions in cons
con_counts = defaultdict(dict)
for method in clean_types:
    col = f'{method}_con_aspect_sentiments'
    for aspect in all_aspects:
        con_counts[aspect][method] = df_sample[col].dropna().apply(lambda x: aspect in x).sum()

# Create DataFrames
pro_df = pd.DataFrame.from_dict(pro_counts, orient='index').fillna(0).astype(int).sort_index()
con_df = pd.DataFrame.from_dict(con_counts, orient='index').fillna(0).astype(int).sort_index()

# Pretty print like your example
print("\nAspect Mentions in PROS:")
print(f"{'Aspect':30}" + "".join([f"{ct:>12}" for ct in clean_types]))
print("-" * (30 + 12 * len(clean_types)))
for aspect, row in pro_df.iterrows():
    print(f"{aspect:30}" + "".join([f"{row[ct]:12}" for ct in clean_types]))

print("\nAspect Mentions in CONS:")
print(f"{'Aspect':30}" + "".join([f"{ct:>12}" for ct in clean_types]))
print("-" * (30 + 12 * len(clean_types)))
for aspect, row in con_df.iterrows():
    print(f"{aspect:30}" + "".join([f"{row[ct]:12}" for ct in clean_types]))



Aspect Mentions in PROS:
Aspect                                 min       lemma    Stopword    Symspell
------------------------------------------------------------------------------
culture                                197         188         188         188
diversity & inclusion                   19          18          18          18
growth & learning                      222         221         221         221
hr & recruitment                       169         165         150         105
job role clarity                        33          32          32          32
job security                            19          19          19          19
management                              81          66          66          66
office & location                       91          87          87          86
pay                                    163         157         157         157
people                                 246         185         185         185
performance feedback      

### Findings:
Reviewing the aspect based sentiment analysis on the 4 differently cleaned pros and cons, we found that the minimal cleaning (removes new lines, strips non ASCII characters) detects the most aspect.
After going through the results manually in the exported csv, we confirmed that the aspect detection on the minimal cleaning best conveys the aspects and sentiments mentioned in the reviews.

## Going forward, will be using the pro_clean_min and con_clean_min in the analysis

### Rerunning on the full sample

### Clean

In [18]:
dfFinal = pd.read_csv('filtered_england_recent.csv')

In [19]:
for col in ['pros', 'cons']:
    print(f"Processing column: {col}")
    dfFinal[f'{col}_clean_min'] = dfFinal[col].progress_apply(clean_min)

Processing column: pros


  0%|          | 0/89427 [00:00<?, ?it/s]

Processing column: cons


  0%|          | 0/89427 [00:00<?, ?it/s]

# !!!!  Warning: adding this line for if we're not running on GPU
### Comment out the line below if running on GPU

In [20]:
#dfFinal = dfFinal.sample(frac=0.1, random_state=20).copy()

### Applying aspet groups

In [21]:
dfFinal['min_pro_aspect_sentiments'] = dfFinal['pros_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
dfFinal['min_con_aspect_sentiments'] = dfFinal['cons_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))

  0%|          | 0/89427 [00:00<?, ?it/s]

  0%|          | 0/89427 [00:00<?, ?it/s]

### Moving to aggregating the aspects per company

In [22]:
def flatten_aspect_sentiments(row, source):
    results = []
    firm = row.get('firm') or row.get('company')  # fallback
    sentiments = row.get(source, {})
    for aspect, scores in sentiments.items():
        # Aggregate actual star rating (1–5)
        rating = scores.get("label")
        all_stars = {i: scores.get(i, 0.0) for i in range(1, 6)}

        results.append({
            "firm": firm,
            "aspect": aspect,
            "source": source.replace('_aspect_sentiments', ''),  # 'pro' or 'con'
            "star_rating": rating,
            **all_stars
        })
    return results

In [23]:
# 3. Apply to full dataset
flattened = []
for _, row in dfFinal.iterrows():
    flattened += flatten_aspect_sentiments(row, 'min_pro_aspect_sentiments')
    flattened += flatten_aspect_sentiments(row, 'min_con_aspect_sentiments')

flat_df = pd.DataFrame(flattened)

# 4. Aggregate star ratings + count mentions per company/aspect
company_aspect_matrix = flat_df.groupby(['firm', 'aspect'])['star_rating'].agg(
    avg_star_rating='mean',
    n_mentions='count'
).reset_index()
company_aspect_pivot = company_aspect_matrix.pivot(
    index='firm',
    columns='aspect',
    values='avg_star_rating'
)

In [24]:
company_aspect_matrix

,firm,aspect,avg_star_rating,n_mentions
0,AFH Wealth Management,culture,2.769231,13
1,AFH Wealth Management,growth & learning,3.444444,9
2,AFH Wealth Management,hr & recruitment,2.521739,23
3,AFH Wealth Management,job role clarity,1.000000,4
4,AFH Wealth Management,job security,1.400000,5
...,...,...,...,...
5196,the LEGO Group,remote work,3.000000,3
5197,the LEGO Group,stress & pressure,2.700000,10
5198,the LEGO Group,teamwork & collaboration,3.538462,39
5199,the LEGO Group,tools & resources,3.428571,14


In [25]:
company_aspect_pivot

aspect,culture,diversity & inclusion,growth & learning,hr & recruitment,job role clarity,job security,management,office & location,pay,people,performance feedback,perks & benefits,remote work,stress & pressure,teamwork & collaboration,tools & resources,work-life balance
firm,,,,,,,,,,,,,,,,,
AFH Wealth Management,2.769231,NaN,3.444444,2.521739,1.000000,1.400000,2.375000,3.142857,1.894737,3.294118,2.500000,3.000000,NaN,2.000000,2.571429,2.111111,3.166667
AJ Bell,3.416667,3.000000,3.250000,3.000000,2.125000,2.800000,2.804878,3.468085,2.428571,3.438596,2.875000,2.862745,4.333333,2.928571,3.290323,3.413793,2.870968
ASOS,3.209302,2.944444,2.550360,2.807692,2.350877,1.400000,1.754464,3.285714,2.335000,3.311787,1.809524,3.685237,3.000000,2.404040,2.701987,2.864407,2.644444
AXA UK,3.543860,4.250000,3.534091,3.192982,3.100000,2.500000,2.781250,3.057143,3.029851,3.704225,2.333333,3.953125,3.125000,1.800000,3.702703,3.454545,3.730769
Abcam,4.153846,4.250000,3.474576,3.513514,3.266667,2.500000,3.530612,3.302632,2.418605,4.119048,3.400000,3.725000,4.750000,3.119048,4.105263,3.833333,4.063830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i Net Solution,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,2.500000,4.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ibis,2.600000,NaN,3.545455,3.100000,NaN,NaN,1.916667,3.500000,1.666667,3.444444,4.000000,3.333333,NaN,2.000000,2.600000,3.000000,2.333333
mydentist UK,3.833333,NaN,3.545455,3.064516,2.166667,3.000000,2.272727,2.666667,2.085714,3.153846,2.250000,2.928571,4.000000,2.666667,4.000000,3.291667,3.222222


### Exporting results

In [26]:
# 6. Export
company_aspect_matrix.to_csv('Final_company_aspect_matrix_with_counts.csv', index=False)
company_aspect_pivot.to_csv('Final_company_aspect_matrix_wide.csv', index=False)
